In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
import pickle

In [2]:
import warnings 

warnings.filterwarnings("ignore") 

In [3]:
with open(r"for_test.pickle", "rb") as input_file:
    for_test=pickle.load(input_file)
lead_time_median = for_test['lead_time_median']
train_columns = for_test['train_columns']
scaler = for_test['scaler']
trained_models = for_test['trained_models']
ensemble_model = for_test['ensemble_model']


In [4]:
def final_fun_1(X):
    X['lead_time'].fillna(lead_time_median,inplace=True)
    X['forecast']=(X['forecast_3_month']>0)|(X['forecast_6_month']>0)|(X['forecast_9_month']>0)
    X['forecast']=X['forecast'].astype(int)
    X['sales']=(X['sales_1_month']>0)|(X['sales_3_month']>0)|(X['sales_6_month']>0)|(X['sales_9_month']>0)
    X['sales']=X['sales'].astype(int)

    X = X.drop(labels=['sku'],axis=1)
    X= pd.get_dummies(X,drop_first=True)

    # https://stackoverflow.com/questions/41335718/keep-same-dummy-variable-in-training-and-testing-data
    # Ensuring same set of columns for train and test
    delta_cols = set(train_columns) - set(X.columns)

    for c in delta_cols:
        X[c] = 0

    X = X[train_columns]
    X_norm = scaler.transform(X)
    X = pd.DataFrame(X_norm,columns=X.columns)
    
    test_x = X.iloc[:,:-1]
    ensemble_x = pd.DataFrame()
    for i,m in enumerate(trained_models):
        pred = m.predict_proba(test_x)[:,1]
        pred = pd.DataFrame(pred,columns=[str(i)])
        ensemble_x = pd.concat([ensemble_x,pred],axis=1)
    predict_y = ensemble_model.predict(ensemble_x)
    return predict_y

In [5]:
def final_fun_2(X,y):
    pred = final_fun_1(X)
    return f1_score(y,pred)

In [6]:
num_rows = 10000
test_data = pd.read_csv("dataset/Kaggle_Test_Dataset_v2.csv")
X = test_data[:num_rows]
y = test_data['went_on_backorder'].replace({'Yes':1,'No':0})[:num_rows]
print("F1 score:",final_fun_2(X,y))

F1 score: 0.2738001573564123
